<a href="https://colab.research.google.com/github/aliceding0924/ModelTest/blob/main/ResNetTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/Model Test")

In [3]:
import torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time

from torchvision import models

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
import torchvision.models.resnet
model = torchvision.models.resnet101()

In [6]:
# print(model)

In [15]:
train_datasets = torchvision.datasets.CIFAR10(
    root = r'../data',
    download=True,
    train=True,
    transform=transforms.ToTensor()
)
train_dataloader= DataLoader(
    dataset=train_datasets,
    batch_size=64
)

test_datasets = torchvision.datasets.CIFAR10(
    root = r'../data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
test_dataloader = DataLoader(
    dataset=test_datasets,
    batch_size=64
)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
train_datasets

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [17]:
train_datasets_size = len(train_datasets)
test_datasets_size = len(test_datasets)
print("训练集数量为：{}".format(train_datasets_size))
print("测试集数量为：{}".format(test_datasets_size))

训练集数量为：50000
测试集数量为：10000


In [18]:
runing_mode = "gpu" # cpu,gpu, gpus
if runing_mode == "gpu" and torch.cuda.is_available():
    print("use cuda")
    device = torch.device("cuda")
else:
    print("use cpu")
    device = torch.device("cpu")

use cuda


In [19]:
if torch.cuda.is_available():
    model.cuda()

In [20]:
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)
learning_rate = 1e-2
optim = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [22]:
epoch = 2
train_step, test_step = 0, 0
for i in range(epoch):
    print("~~~~~~~~~~~~第{}轮训练开始~~~~~~~~~~~".format(i+1))
    start = time.time()
    model.train()
    for data in train_dataloader:
        imgs, targets = data
        imgs, targets = imgs.to(device), targets.to(device)
        output = model(imgs)
        loss = loss_fn(output, targets)

        optim.zero_grad()
        loss.backward()
        optim.step()

        train_step += 1
        if train_step % 200 == 0:
            print("第{}次训练，loss={:.3f}".format(train_step, loss.item()))
    #
    model.eval()
    with torch.no_grad():
        test_loss, true_num = 0, 0
        for data in test_dataloader:
            imgs, targets = data
            imgs, targets = imgs.to(device), targets.to(device)
            output = model(imgs)
            test_loss += loss_fn(output, targets)
            true_num += (output.argmax(1) == targets).sum()
    end = time.time()
    print("第{}轮测试集上的loss:{:.3f}, 正确率为:{:.3f}%,耗时:{:.3f}".format(test_step+1, test_loss.item(), 100 * true_num / test_datasets_size, end-start))
    test_step += 1

~~~~~~~~~~~~第1轮训练开始~~~~~~~~~~~
第200次训练，loss=2.758
第400次训练，loss=2.437
第600次训练，loss=1.815
第1轮测试集上的loss:386.034, 正确率为:23.940%,耗时:281.865
~~~~~~~~~~~~第2轮训练开始~~~~~~~~~~~
第800次训练，loss=1.782
第1000次训练，loss=1.622
第1200次训练，loss=1.711
第1400次训练，loss=1.555
第2轮测试集上的loss:324.244, 正确率为:32.250%,耗时:282.274
